The aim of this nb is to populate a postgresdatabase with Forex data from Darwinex

In [1]:
import os

%pip install psycopg2
import psycopg2

def open_connection(database="darwinex3"):
    #establishing the connection
    conn = psycopg2.connect(
            database=database,
            user='postgres', password='Nocturno1357+', host='127.0.0.1', port= '5432'
    )
    conn.autocommit = True
    return conn

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Add code to connect Darwinex FTP and download the missing days


In [3]:
def create_extension_timescaledb(dbname):
    conn = open_connection(database=dbname)
    sql = '''create extension if not exists timescaledb;'''
    cursor.execute(sql)
    print("Configuring timescaledb")
    conn.commit()
    conn.close()

In [4]:
# This depends on the timeseriesdb timescale.com
# installation docs at https://docs.timescale.com/install/latest/

def create_table(pair,side,database,drop=False):
    conn = open_connection(database=database)
    cursor = conn.cursor()
    table_name = f"d_{pair}_{side}"
    # Avoid droping tables by mistake
    if drop:
        cursor.execute(f"DROP TABLE IF EXISTS {table_name}")        
        print(f"Delete created successfully........for {table_name}")
    
    # TODO: If table does not exists
    sql =f'''CREATE TABLE d_{pair}_{side}(
       TIMESTAMP TIMESTAMP NOT NULL,
       DATETIME VARCHAR(15),
       {side} FLOAT NOT NULL,
       VOL FLOAT
    )'''
    cursor.execute(sql)    
    print(f"Table {table_name} created successfully........")    
    # create hypertable
    sql=f'''select create_hypertable('{table_name}','timestamp');'''
    cursor.execute(sql)
    print(f"Table {table_name} created hypertable........")
    conn.close()


In [ ]:
# TODO

database = "darwinex3"
pairs=["usdchf"]

for pair in pairs:
    side="ask"
    create_table(pair,side,database)
    side="bid"
    create_table(pair,side,database)


In [5]:
from datetime import datetime

def get_datetime(fname):
    date = fname.split('_')[2]
    time =  fname.split('_')[3].split('.')[0]
    #print(date,time)
    return date+'_'+time

def unzip_log_file(full_path):
    import gzip    
    registers=''
    try:
        with gzip.open(full_path,'rb') as f:
            file_content = f.read()
        registers = file_content.decode('ascii').split("\n")        
    except OSError:
        print(OSError.errno)
        print(f"Error unzipping {full_path} | ")
        return None
    return registers
    
def launch_sql_insert_query(values, table_name, side):    
    sql=None
    try:
        jvalues = ",".join(values)     
        sql = f'''INSERT INTO {table_name}(timestamp,datetime,{side},vol) VALUES {jvalues}'''    
        conn = open_connection()
        cursor = conn.cursor()
        cursor.execute(sql)        
        conn.commit()
        conn.close()
    except:
        #print("sql query",sql)
        print(f"Problem found inserting into table {table_name}")
        return "KO"

def collect_csv_registers(registers,datehour):
    # collect from csv resisters    
            
    values = []
    for reg in registers:
        if reg == '':
            continue
            
        sreg = reg.split(',')
        
        #print(sreg)
        
        try:
            timestamp = datetime.fromtimestamp(float(float(sreg[0])/1000))
            
            price     = float(sreg[1]) 
            vol       = int(float(sreg[2]))
        
            sreg_for_db = f"('{timestamp}','{datehour}','{price}','{vol}')"
        
            #print(sreg_for_db)
        
            values.append(sreg_for_db)
        except ValueError:
            print("Problem converting data from registers",sreg)
    return values
    
def insert_into_db(side,pair,fname,path):

    full_path = path+"\\"+fname
    
    registers = unzip_log_file(full_path)

    if registers == None:
        return "KO problem found extrating resigers"
    
    datehour = get_datetime(fname)
    
    values = collect_csv_registers(registers,datehour)

    table_name = f"d_{pair}_{side}"
    
    launch_sql_insert_query(values, table_name, side)    
        
    return "OK commit "+fname


def insert_bulk_pair(root,pair,side, resume_at=None):
    import os
    path = os.path.join(root,pair.upper())    
    lfiles = os.listdir(path)
    assert len(lfiles) > 0
    
    index_to_resume = None
    if resume_at != None:
        index_to_resume = resume_at.get(pair,None)

    #print(index_to_resume)
    for index, fname in enumerate(lfiles):
     
        if index_to_resume!=None and index_to_resume > index:
            #print(f"{index} skip file {fname}")
            continue            
        
        if not fname.endswith(".log.gz"):
            continue
        
        if side.upper() not in fname:
            continue
        
        is_ok = insert_into_db(side,pair,fname,path)
        if(is_ok =="KO"):
            print("something went wrong",index)
            break
        print(datetime.now(), index, is_ok)
        
        
def multiple_inserts(root, pairs, sides, resume_at=None):
    for pair in pairs: 
        for side in sides:
            insert_bulk_pair(root,pair,side, resume_at)

In [6]:
root = 'C:\\Users\\ruben\\darwinex_data'
resume_at = {}
resume_at["eurgbp"] = 15449 # Resume ask at 22880
pairs  = ['eurgbp','usdchf'] 
sides  = ['ask','bid'] 
multiple_inserts(root,pairs, sides, resume_at)

2022-04-29 13:25:31.588327 15449 OK commit EURGBP_ASK_2020-03-26_12.log.gz
2022-04-29 13:25:32.285254 15450 OK commit EURGBP_ASK_2020-03-26_13.log.gz
2022-04-29 13:25:32.818253 15451 OK commit EURGBP_ASK_2020-03-26_14.log.gz
2022-04-29 13:25:33.289241 15452 OK commit EURGBP_ASK_2020-03-26_15.log.gz
2022-04-29 13:25:33.693260 15453 OK commit EURGBP_ASK_2020-03-26_16.log.gz
2022-04-29 13:25:33.996260 15454 OK commit EURGBP_ASK_2020-03-26_17.log.gz
2022-04-29 13:25:34.220256 15455 OK commit EURGBP_ASK_2020-03-26_18.log.gz
2022-04-29 13:25:34.586248 15456 OK commit EURGBP_ASK_2020-03-26_19.log.gz
2022-04-29 13:25:34.750245 15457 OK commit EURGBP_ASK_2020-03-26_20.log.gz
2022-04-29 13:25:34.807238 15458 OK commit EURGBP_ASK_2020-03-26_21.log.gz
2022-04-29 13:25:34.957242 15459 OK commit EURGBP_ASK_2020-03-26_22.log.gz
2022-04-29 13:25:35.151256 15460 OK commit EURGBP_ASK_2020-03-26_23.log.gz
2022-04-29 13:25:35.270329 15461 OK commit EURGBP_ASK_2020-03-27_00.log.gz
2022-04-29 13:25:35.42064

2022-04-29 13:25:59.989363 15561 OK commit EURGBP_ASK_2020-04-02_04.log.gz
2022-04-29 13:26:00.094849 15562 OK commit EURGBP_ASK_2020-04-02_05.log.gz
2022-04-29 13:26:00.313849 15563 OK commit EURGBP_ASK_2020-04-02_06.log.gz
2022-04-29 13:26:00.555853 15564 OK commit EURGBP_ASK_2020-04-02_07.log.gz
2022-04-29 13:26:00.777853 15565 OK commit EURGBP_ASK_2020-04-02_08.log.gz
2022-04-29 13:26:00.997855 15566 OK commit EURGBP_ASK_2020-04-02_09.log.gz
2022-04-29 13:26:01.215859 15567 OK commit EURGBP_ASK_2020-04-02_10.log.gz
2022-04-29 13:26:01.449852 15568 OK commit EURGBP_ASK_2020-04-02_11.log.gz
2022-04-29 13:26:01.786856 15569 OK commit EURGBP_ASK_2020-04-02_12.log.gz
2022-04-29 13:26:02.196860 15570 OK commit EURGBP_ASK_2020-04-02_13.log.gz
2022-04-29 13:26:02.787850 15571 OK commit EURGBP_ASK_2020-04-02_14.log.gz
2022-04-29 13:26:03.209858 15572 OK commit EURGBP_ASK_2020-04-02_15.log.gz
2022-04-29 13:26:03.507851 15573 OK commit EURGBP_ASK_2020-04-02_16.log.gz
2022-04-29 13:26:03.70284

2022-04-29 13:26:19.392816 15672 OK commit EURGBP_ASK_2020-04-08_18.log.gz
2022-04-29 13:26:19.492804 15673 OK commit EURGBP_ASK_2020-04-08_19.log.gz
2022-04-29 13:26:19.552795 15674 OK commit EURGBP_ASK_2020-04-08_20.log.gz
2022-04-29 13:26:19.637814 15675 OK commit EURGBP_ASK_2020-04-08_21.log.gz
2022-04-29 13:26:19.704816 15676 OK commit EURGBP_ASK_2020-04-08_22.log.gz
2022-04-29 13:26:19.772351 15677 OK commit EURGBP_ASK_2020-04-08_23.log.gz
2022-04-29 13:26:19.867767 15678 OK commit EURGBP_ASK_2020-04-09_00.log.gz
2022-04-29 13:26:19.975776 15679 OK commit EURGBP_ASK_2020-04-09_01.log.gz
2022-04-29 13:26:20.050767 15680 OK commit EURGBP_ASK_2020-04-09_02.log.gz
2022-04-29 13:26:20.129814 15681 OK commit EURGBP_ASK_2020-04-09_03.log.gz
2022-04-29 13:26:20.198081 15682 OK commit EURGBP_ASK_2020-04-09_04.log.gz
2022-04-29 13:26:20.268051 15683 OK commit EURGBP_ASK_2020-04-09_05.log.gz
2022-04-29 13:26:20.402043 15684 OK commit EURGBP_ASK_2020-04-09_06.log.gz
2022-04-29 13:26:20.60005

2022-04-29 13:26:31.655925 15783 OK commit EURGBP_ASK_2020-04-15_09.log.gz
2022-04-29 13:26:31.849930 15784 OK commit EURGBP_ASK_2020-04-15_10.log.gz
2022-04-29 13:26:31.959925 15785 OK commit EURGBP_ASK_2020-04-15_11.log.gz
2022-04-29 13:26:32.166928 15786 OK commit EURGBP_ASK_2020-04-15_12.log.gz
2022-04-29 13:26:32.337926 15787 OK commit EURGBP_ASK_2020-04-15_13.log.gz
2022-04-29 13:26:32.569931 15788 OK commit EURGBP_ASK_2020-04-15_14.log.gz
2022-04-29 13:26:32.751928 15789 OK commit EURGBP_ASK_2020-04-15_15.log.gz
2022-04-29 13:26:32.907447 15790 OK commit EURGBP_ASK_2020-04-15_16.log.gz
2022-04-29 13:26:33.030098 15791 OK commit EURGBP_ASK_2020-04-15_17.log.gz
2022-04-29 13:26:33.161097 15792 OK commit EURGBP_ASK_2020-04-15_18.log.gz
2022-04-29 13:26:33.276091 15793 OK commit EURGBP_ASK_2020-04-15_19.log.gz
2022-04-29 13:26:33.366126 15794 OK commit EURGBP_ASK_2020-04-15_20.log.gz
2022-04-29 13:26:33.419610 15795 OK commit EURGBP_ASK_2020-04-15_21.log.gz
2022-04-29 13:26:33.47461

2022-04-29 13:26:44.931953 15894 OK commit EURGBP_ASK_2020-04-22_00.log.gz
2022-04-29 13:26:44.999957 15895 OK commit EURGBP_ASK_2020-04-22_01.log.gz
2022-04-29 13:26:45.073953 15896 OK commit EURGBP_ASK_2020-04-22_02.log.gz
2022-04-29 13:26:45.129658 15897 OK commit EURGBP_ASK_2020-04-22_03.log.gz
2022-04-29 13:26:45.189716 15898 OK commit EURGBP_ASK_2020-04-22_04.log.gz
2022-04-29 13:26:45.254630 15899 OK commit EURGBP_ASK_2020-04-22_05.log.gz
2022-04-29 13:26:45.376615 15900 OK commit EURGBP_ASK_2020-04-22_06.log.gz
2022-04-29 13:26:45.516622 15901 OK commit EURGBP_ASK_2020-04-22_07.log.gz
2022-04-29 13:26:45.642615 15902 OK commit EURGBP_ASK_2020-04-22_08.log.gz
2022-04-29 13:26:45.761219 15903 OK commit EURGBP_ASK_2020-04-22_09.log.gz
2022-04-29 13:26:45.856217 15904 OK commit EURGBP_ASK_2020-04-22_10.log.gz
2022-04-29 13:26:45.986222 15905 OK commit EURGBP_ASK_2020-04-22_11.log.gz
2022-04-29 13:26:46.095217 15906 OK commit EURGBP_ASK_2020-04-22_12.log.gz
2022-04-29 13:26:46.24621

...

Problem found inserting into table d_eurgbp_bid
2022-04-29 14:18:47.270562 46108 OK commit EURGBP_BID_2020-07-29_18.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:18:47.467554 46109 OK commit EURGBP_BID_2020-07-29_19.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:18:47.649999 46110 OK commit EURGBP_BID_2020-07-29_20.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:18:47.705386 46111 OK commit EURGBP_BID_2020-07-29_21.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:18:47.757590 46112 OK commit EURGBP_BID_2020-07-29_22.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:18:47.818869 46113 OK commit EURGBP_BID_2020-07-29_23.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:18:47.930037 46114 OK commit EURGBP_BID_2020-07-30_00.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:18:48.029046 46115 OK commit EURGBP_BID_2020-07-30_01.log.gz
Problem found in

Problem found inserting into table d_eurgbp_bid
2022-04-29 14:18:54.491169 46175 OK commit EURGBP_BID_2020-08-03_13.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:18:54.632169 46176 OK commit EURGBP_BID_2020-08-03_14.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:18:54.740171 46177 OK commit EURGBP_BID_2020-08-03_15.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:18:54.817756 46178 OK commit EURGBP_BID_2020-08-03_16.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:18:55.003000 46179 OK commit EURGBP_BID_2020-08-03_17.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:18:55.079991 46180 OK commit EURGBP_BID_2020-08-03_18.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:18:55.146007 46181 OK commit EURGBP_BID_2020-08-03_19.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:18:55.198997 46182 OK commit EURGBP_BID_2020-08-03_20.log.gz
Problem found in

Problem found inserting into table d_eurgbp_bid
2022-04-29 14:19:00.727750 46242 OK commit EURGBP_BID_2020-08-06_08.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:19:00.837750 46243 OK commit EURGBP_BID_2020-08-06_09.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:19:00.940740 46244 OK commit EURGBP_BID_2020-08-06_10.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:19:01.052739 46245 OK commit EURGBP_BID_2020-08-06_11.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:19:01.181730 46246 OK commit EURGBP_BID_2020-08-06_12.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:19:01.421540 46247 OK commit EURGBP_BID_2020-08-06_13.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:19:01.597548 46248 OK commit EURGBP_BID_2020-08-06_14.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:19:01.830820 46249 OK commit EURGBP_BID_2020-08-06_15.log.gz
Problem found in

Problem found inserting into table d_eurgbp_bid
2022-04-29 14:19:07.246023 46310 OK commit EURGBP_BID_2020-08-11_04.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:19:07.312025 46311 OK commit EURGBP_BID_2020-08-11_05.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:19:07.401045 46312 OK commit EURGBP_BID_2020-08-11_06.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:19:07.500033 46313 OK commit EURGBP_BID_2020-08-11_07.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:19:07.617038 46314 OK commit EURGBP_BID_2020-08-11_08.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:19:07.738029 46315 OK commit EURGBP_BID_2020-08-11_09.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:19:07.848038 46316 OK commit EURGBP_BID_2020-08-11_10.log.gz
Problem found inserting into table d_eurgbp_bid
2022-04-29 14:19:07.979034 46317 OK commit EURGBP_BID_2020-08-11_11.log.gz
Problem found in

KeyboardInterrupt: 

In [ ]:
# this is the way to ingest ms into postgres
#datetime.fromtimestamp(float('1506891900032')/1000)

#conn = open_connection()
#fname='USDCAD_ASK_2020-02-28_19.log.gz'
#pair='usdcad'
#insert_into_db(side,pair,fname,conn)

#fname=path+'\\USDCAD_ASK_2020-02-28_19.log.gz'
#with gzip.open(full_path,'rb') as f:
#    file_content = f.read()
#print(file_content)